## PageRank on Wikipedia

### Prerequisites

1. Download a dump of Wikipedia's articles, named `enwiki-{date_string}-pages-articles-multistream.xml.bz2`
2. Download the `enwiki-{date_string}-pages-articles-multistream-index.txt.bz2` file
3. Move those files into the same folder, removing the `enwiki-{date_string}` prefix
4. Process the `xml.bz2` file into a Parquet file using `wikiplain.load_bz2`

In [2]:
import asyncio
import glob
import gzip
import io
import itertools
import json
import math
import operator
import os
import pickle
import random
import re
import shutil
import socket
import struct
import subprocess
import sys
import tarfile
import time
from collections import ChainMap, defaultdict, deque
from contextlib import asynccontextmanager
from dataclasses import dataclass
from datetime import datetime
from enum import Enum, auto
from functools import lru_cache, partial
from urllib.parse import urlencode, urlsplit, quote as urlquote, unquote as urlunquote
from typing import Any, Awaitable, Callable, Literal, TypeVar

import cbor2
import numpy as np
import pypocketmap as pkm
import pyarrow.parquet as pq
import polars as pl
import sqlalchemy as sa
import scipy.sparse
from dotenv import load_dotenv
from ipywidgets import interact
from sqlalchemy import create_engine
from sqlalchemy.sql import select, text as sqltext
from tqdm.auto import tqdm
from arsenal.datastructures.unionfind import UnionFind

import wikiplain
from nbhelpers.polars import pager, searcher

In [3]:
load_dotenv()

True

In [4]:
pl.Config.set_fmt_str_lengths(160)

polars.config.Config

In [5]:
class PageRankFiles:
    def __init__(self, date_string):
        self.date_string = date_string
        self.enwiki_dir = f"{os.environ['ENWIKI_DIR']}/{date_string}"
        self.parquet_dir = os.environ.get('ENWIKI_PARQUET_DIR', self.enwiki_dir)
        try:
            os.mkdir(f"{self.enwiki_dir}/pagerank")
        except FileExistsError:
            pass
    
    @property
    def enwiki_parquet_filename(self):
        return f"{self.parquet_dir}/enwiki_{self.date_string}.parquet"
    
    @property
    def pagerank_parquet_filename(self):
        return f"{self.parquet_dir}/enwiki_{self.date_string}_pagerank.parquet"

    @property
    def nub_filename(self):
        return f"{self.enwiki_dir}/pagerank/nub.bin"
    
    @property
    def id_map_filename(self):
        return f"{self.enwiki_dir}/pagerank/id_map.bin"
    
    @property
    def id_map2_filename(self):
        return f"{self.enwiki_dir}/pagerank/id_map2.bin"
    
    @property
    def dense_id_arr_filename(self):
        return f"{self.enwiki_dir}/pagerank/dense_id_arr.npy"
    
    @property
    def edge_filename_pattern(self):
        return f"{self.enwiki_dir}/pagerank/edges_*.npz"
    
    def edge_filenames(self, num_partitions):
        return [
            f"{self.enwiki_dir}/pagerank/edges_{i}.npz"
            for i in range(num_partitions)
        ]

    @property
    def in_degree_filename(self):
        return f"{self.enwiki_dir}/pagerank/in_degree.npy"
    
    @property
    def out_degree_filename(self):
        return f"{self.enwiki_dir}/pagerank/out_degree.npy"
    
    def adjacency_filename(self, partition):
        return f"{self.enwiki_dir}/pagerank/adjacency_{partition}.npz"
    
    def adjacency_filenames(self, num_partitions):
        return [self.adjacency_filename(i) for i in range(num_partitions)]

In [6]:
files = PageRankFiles("20240401")

### Find title collisions

1. There are some pages with the same title - I think this is caused by pages deleted and recreated while the snapshot is in progress

In [7]:
pqf = pq.ParquetFile(files.enwiki_parquet_filename)

In [8]:
def get_overwritten():
    overwritten = set()
    timestamp_map = {}
    article_ids = {}
    pqf_size = 0
    for batch in tqdm(pqf.iter_batches(batch_size=100), total=pqf.num_row_groups):
        for aid, ns, ttl, tm in zip(batch["id"].to_numpy(), batch["ns"].to_numpy(), batch["title"].to_pylist(), batch["timestamp"].to_pylist()):
            pqf_size += 1
            if ns != 0:
                continue
            tm = np.datetime64(tm)
            other_id = article_ids.setdefault(ttl, aid)
            if other_id != aid:
                if (timestamp_map[ttl], other_id) < (tm, aid):
                    print(f"{ttl!r}: {aid} > {other_id}")
                    overwritten.add(other_id)
                    article_ids[ttl] = aid
                    timestamp_map[ttl] = tm
                else:
                    print(f"{ttl!r}: {other_id} > {aid}")
                    overwritten.add(aid)
            else:
                timestamp_map[ttl] = tm
    return overwritten, pqf_size

try:
    with open(files.nub_filename, "rb") as fp:
        overwritten, pqf_size = cbor2.load(fp)
except Exception:
    overwritten, pqf_size = get_overwritten()
    overwritten = {int(e) for e in overwritten}
    with open(files.nub_filename, "wb") as fp:
        cbor2.dump((overwritten, pqf_size), fp)

#### get_overwritten time to complete

| date | batches/s | time |
| :--- | :------ | :--- |
| 2024-05-14 | 230.5 | 17min 04s

### Build representation of articles/links as a graph

1. Create `id_map` from non-redirecting article titles to node number, and `id_map2` from redirecting article titles to node number
2. Use `wikiplain` to extract link titles, and use above maps to convert to (src_id, dest_id) pairs

In [9]:
class Vec:
    def __init__(self, dtype):
        self.array = np.ndarray((1024,), dtype=dtype)
        self.length = 0
    
    @property
    def capacity(self):
        return self.array.shape[0]

    def append(self, v):
        idx = self.length
        if idx >= self.capacity:
            addsz = max(2, self.capacity)
            self.array = np.hstack((self.array, np.zeros((addsz,), dtype=self.array.dtype)))
        self.array[idx] = v
        self.length += 1

In [10]:
def iterdecode(f):
    decoder = cbor2.CBORDecoder(f)
    while True:
        try:
            yield decoder.decode()
        except EOFError:
            return

In [11]:
def get_id_maps():
    redirect_group_map = UnionFind()
    id_map = pkm.create(str, int)
    redirect_lst = []
    dense_ids = Vec(dtype=np.int64)
    for batch in tqdm(pqf.iter_batches(batch_size=100), total=math.ceil(pqf_size / 100)):
        for aid, ns, ttl, redir in zip(batch["id"].to_numpy(), batch["ns"].to_numpy(), batch["title"].to_pylist(), batch["redirect"].to_pylist()):
            if ns != 0 or aid in overwritten:
                continue
            if redir is not None:
                redirect_group_map.union(ttl, redir)
                redirect_lst.append(ttl)
            else:
                assert ttl not in id_map, f"Expected unique titles, got second instance of {ttl}"
                dense_ids.append(aid)
                id_map[ttl] = len(id_map)
    id_map2 = pkm.create(str, int)
    for group in redirect_group_map.classes():
        centers = [ttl for ttl in group if ttl in id_map]
        if len(centers) == 0:
            continue
        assert len(centers) == 1, str(centers)
        for ttl in group:
            if ttl != centers[0]:
                id_map2[ttl] = id_map[centers[0]]
    return id_map, id_map2, dense_ids.array[:dense_ids.length]

try:
    with open(files.id_map_filename, "rb") as fp:
        id_map = pkm.create(str, int)
        for k, v in iterdecode(fp):
            id_map[k] = v
    with open(files.id_map2_filename, "rb") as fp:
        id_map2 = pkm.create(str, int)
        for k, v in iterdecode(fp):
            id_map2[k] = v
    with open(files.dense_id_arr_filename, "rb") as fp:
        dense_id_arr = np.load(fp)
except Exception:
    id_map, id_map2, dense_id_arr = get_id_maps()
    with open(files.id_map_filename, "wb") as fp:
        for k, v in id_map.items():
            cbor2.dump((k, v), fp)
    with open(files.id_map2_filename, "wb") as fp:
        for k, v in id_map2.items():
            cbor2.dump((k, v), fp)
    with open(files.dense_id_arr_filename, "wb") as fp:
        np.save(fp, dense_id_arr)

In [12]:
len(id_map), len(id_map2)

(6806227, 10873732)

In [13]:
list(itertools.islice(iter(id_map.keys()), 10))

['Martín Vázquez',
 'Rouverol',
 'Ron Hansell',
 'Jim Wolf (musician)',
 'McAllen Miller International Airport',
 '1937 Albanian National Championship',
 'Manilius (crater)',
 'Nerka Lake',
 'Metallacarboxylic acid',
 'Nicolas Appert']

In [14]:
list(itertools.islice(iter(id_map2.keys()), 10))

['Office for fair access',
 'The Catalyst (newspaper)',
 'Scarecrow Press historical dictionary series',
 'Boissieri',
 'Edward Northey (disambiguation)',
 'Herzogschloss Zweibrücken',
 'Trinity Square (disambiguation)',
 'Urban cowboy',
 'K 424a',
 'Nawaz Shareef']

In [15]:
class PairVec:
    def __init__(self, dtype):
        self.array = np.ndarray((1024, 2), dtype=dtype)
        self.length = 0
    
    @property
    def capacity(self):
        return self.array.shape[0]

    def append(self, v1, v2):
        idx = self.length
        if idx >= self.capacity:
            addsz = max(2, self.capacity)
            self.array = np.vstack((self.array, np.zeros((addsz, 2), dtype=self.array.dtype)))
        self.array[idx] = [v1, v2]
        self.length += 1

In [44]:
def parse_wiki_link(line):
    dest_ttl = line.strip()
    if len(dest_ttl) == 0:
        return None
    dest_ttl = dest_ttl[0].upper() + dest_ttl[1:]
    dest_ttl = dest_ttl.split('|', maxsplit=1)[0]
    dest_ttl = dest_ttl.split('#', maxsplit=1)[0]
    return dest_ttl

In [17]:
LOG_PARTITION_SIZE = 16
PARTITION_SIZE = 1 << LOG_PARTITION_SIZE
N = len(id_map)
NUM_PARTITIONS = math.ceil(N / PARTITION_SIZE)

In [18]:
combined_id_map = ChainMap(id_map, id_map2)

In [19]:
N

6806227

### Edge format

- `edges_{n}.npz` stores the outgoing links from `PARITION_SIZE*n ..< PARTITION_SIZE*(n+1)`
- These are stored in a list where element `i` contains the links out to `PARITION_SIZE*i ..< PARTITION_SIZE*(i+1)`

In [20]:
def chunk(iterable, size):
    """Split an iterable into list chunks of size `n`.
    
    The last chunk can be fewer than `n` elements long, but it won't be empty.
    """
    iterator = iter(iterable)
    while True:
        chunk = list(itertools.islice(iterator, size))
        if chunk:
            yield chunk
        else:
            return

def lazy_chunk(iterable, n):
    """Split an iterable into iterable chunks of size `n`.
    
    The last chunk can be fewer than `n` elements long, but it won't be empty.
    """
    iterator = iter(iterable)
    while True:
        try:
            first = next(iterator)
        except StopIteration:
            return
        yield itertools.chain([first], itertools.islice(iterator, n - 1))

In [57]:
ORDER_TAG_BITS = 3
NUM_ORDER_TAGS = 1 << ORDER_TAG_BITS
DEST_ID_BITS = 31 - ORDER_TAG_BITS
DEST_ID_MASK = (1 << DEST_ID_BITS) - 1
def get_edges():
    in_degree = np.zeros(N, dtype=np.int32)
    out_degree = np.zeros(N, dtype=np.int32)
    with tqdm(position=1, miniters=1000) as progress:
        iterator = tqdm(pqf.iter_batches(batch_size=100), total=math.ceil(pqf_size / 100))
        iterator = map(
            lambda b: zip(
                b["id"].to_numpy(),
                b["ns"].to_numpy(),
                map(operator.attrgetter("is_valid"), b["redirect"]),
                b["text"].to_pylist()
            ),
            iterator
        )
        iterator = itertools.chain.from_iterable(iterator)
        iterator = filter(lambda e: not e[2] and e[1] == 0 and e[0] not in overwritten, iterator)
        iterator = enumerate(map(operator.itemgetter(3), iterator))
        filenames = files.edge_filenames(NUM_PARTITIONS)
        order_tags = np.arange(NUM_ORDER_TAGS, dtype=np.int32)[::-1] << DEST_ID_BITS
        for part_idx, subitr in enumerate(lazy_chunk(iterator, PARTITION_SIZE)):
            edges = [PairVec('int32') for _ in range(0, N, PARTITION_SIZE)]
            for src_id, text in subitr:
                link_idx = 0
                for link in wikiplain.get_links(text):
                    dest_ttl = parse_wiki_link(link)
                    if dest_ttl:
                        dest_id = id_map.get(dest_ttl)
                        dest_id = dest_id if (dest_id is not None) else id_map2.get(dest_ttl)
                        if dest_id is not None:
                            partition = dest_id >> LOG_PARTITION_SIZE
                            edges[partition].append(src_id, order_tags[link_idx] | dest_id)
                            in_degree[dest_id] += 1
                            out_degree[src_id] += 1
                            progress.update()
                            link_idx = min(NUM_ORDER_TAGS - 1, link_idx + 1)
            with open(filenames[part_idx], "wb") as fp:
                np.savez(fp, *([vec.array[:vec.length] for vec in edges]))
    return in_degree, out_degree

In [58]:
edge_fnames = glob.glob(files.edge_filename_pattern)
try:
    assert set(edge_fnames) == set(files.edge_filenames(NUM_PARTITIONS))
    with open(files.in_degree_filename, "rb") as fp:
        in_degree = np.load(fp)
    with open(files.out_degree_filename, "rb") as fp:
        out_degree = np.load(fp)
    # for fname in edge_fnames:
    #    with open(fname, "rb") as fp:
    #         assert len(pickle.load(fp)) == NUM_PARTITIONS
except Exception as exc:
    print(exc)
    in_degree, out_degree = get_edges()
    edge_fnames = glob.glob(files.edge_filename_pattern)
    with open(files.in_degree_filename, "wb") as fp:
        np.save(fp, in_degree)
    with open(files.out_degree_filename, "wb") as fp:
        np.save(fp, out_degree)

0it [00:00, ?it/s]

  0%|          | 0/236033 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### get_edges time to complete

| date | links/s | time |
| :--- | :------ | :--- |
| 2024-02-06 | 101000 | 46min 48s

In [21]:
# def get_dab_array():
#     result = np.zeros(N, dtype=np.bool8)
#     dab_proc = subprocess.Popen(
#         ["wikiplain", "--fraction", "1", "-c", "only-dab", "--ns", "0", files.enwiki_database_filename],
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE
#     )
#     iterator = make_links_iter(dab_proc.stdout)
#     iterator = tqdm(iterator, position=0, total=len(id_map))
#     iterator = map(lambda pair: (pair[0].decode("utf-8"), pair[1]), iterator)
#     for n, subitr in enumerate(lazy_chunk(iterator, PARTITION_SIZE)):
#         for ttl, text in subitr:
#             src_id = id_map[ttl]
#             if len(text) > 0:
#                 result[src_id] = True
#     return result

In [22]:
# LDF = pl.scan_parquet(files.enwiki_parquet_filename)

In [24]:
# try:
#     with open(files.dab_array_filename, "rb") as fp:
#         dab_array = pickle.load(fp)
# except Exception as exc:
#     print(exc)
#     dab_array = get_dab_array()
#     with open(files.dab_array_filename, "wb") as fp:
#         pickle.dump(dab_array, fp)

In [44]:
def compute_adjacency_matrix_slice(partition, progress):
    """Computes the slice of the adjacency matrix A starting at row p*S and ending before row (p+1)*S
    
    p=partition, S=PARTITION_SIZE, and A is defined so that
    A @ np.eye(N)[i] = v, a probability vector where
        v[j] = out-degree(i) > 0 | count((i,j) in E) / out-degree(i)
               otherwise         | 0
    """
    origin_row = partition * PARTITION_SIZE
    n_rows = min(PARTITION_SIZE, N - origin_row)
    index_arrs = []
    value_arrs = []
    pkey = f'arr_{partition}'
    for fname in glob.glob(files.edge_filename_pattern):
        with np.load(fname) as npz:
            vec = npz[pkey]
        vec[:, 1] &= DEST_ID_MASK  # remove order_tag
        # vec is
        #  [[src_id_0, dest_id_0],
        #   [src_id_1, dest_id_1],
        #   ...
        #  ]
        # Sort by (src,dest), make unique and get counts
        key_arr = (vec[:, 0].astype('int64') << 32) | vec[:, 1]
        _, order, count = np.unique(key_arr, return_index=True, return_counts=True)
        vec = vec[order]
        # Normalize `count` based on (src,)
        count = count.astype('float64') / out_degree[vec[:, 0]]
        index_arrs.append(vec)
        value_arrs.append(count)
        progress.update()
    index_arr = np.vstack(index_arrs)
    matrix_slice = scipy.sparse.csr_array(
        (np.hstack(value_arrs), (index_arr[:, 1] - origin_row, index_arr[:, 0])),
        shape=(n_rows, N),
        dtype=np.float64
    )
    return matrix_slice

In [45]:
with tqdm(total=NUM_PARTITIONS**2) as progress:
    for partition in range(NUM_PARTITIONS):
        adj_matrix = compute_adjacency_matrix_slice(partition, progress)
        scipy.sparse.save_npz(files.adjacency_filename(partition), adj_matrix, compressed=False)

  0%|          | 0/10816 [00:00<?, ?it/s]

In [47]:
scipy.sparse.save_npz?

In [27]:
np.quantile(out_degree, [0, 0.1, 0.5, 0.9, 0.99, 0.999, 1]).astype(int)

array([    0,     5,    19,    84,   406,  1189, 16248])

In [28]:
log_out_degree = np.log(out_degree + 2)

In [29]:
log_out_degree /= log_out_degree.sum()

### Global PageRank

The initial rank is a column vector $\mathbf{r}$ where $\mathbf{r}_i = \frac{1}{N}$

The transition matrix $\mathbf{M}$ is N x N; each column represents a source, and each row represents a destination.
$\mathbf{M}_{ij} = P(\text{next}=i\,|\,\text{current}=j)$. Each column **must** sum to 1 for the calculation to be stable, so if page $j$ contains no links, it is treated as if it had a link to every page.

The power method iteratively computes better ranks: $\mathbf{r'} = (1 - \alpha) \mathbf{M}\mathbf{r} + \frac{\alpha}{N}$

### Personalized PageRank

Personalized PageRank uses a preference vector $\mathbf{p}$ in place of the uniform $\frac{1}{N}$ for _teleportation_. Pages with no out-links still use a uniform distribution. The initial rank can be any vector, because of the converging property of the power method (explanation at https://mathworld.wolfram.com/Eigenvector.html)

### Ending iteration

At each iteration, we calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. The initial guess is at maximum entropy, so the first iteration causes perplexity to decrease. Later iterations may change perplexity in either direction; we stop when the change is below a certain threshold.

In [30]:
def perplexity(distribution):
    return np.power(2, np.sum(-distribution * np.log2(distribution)))

def personalized_page_rank(preference, threshold=1, random_jump_prob=0.15):
    current_rank = np.ones(N, dtype=np.float64) / N
    next_rank = np.zeros(N, dtype=np.float64)
    # iteratively update current_rank
    edge_follow_prob = 1 - random_jump_prob
    prev_perplexity = float('inf')
    current_perplexity = perplexity(current_rank)
    current_iter = 0
    iter_start = time.time()
    print("Itr# | ΔPerplexity     | Seconds")
    while abs(prev_perplexity - current_perplexity) > threshold:
        current_iter += 1
        next_rank[:] = random_jump_prob * preference
        # update destinations from non-sink nodes (N x N times N x 1 -> N x 1)
        spread_probs = np.vstack([
            adjacency_matrix_slice.dot(current_rank[:, np.newaxis])
            for adjacency_matrix_slice in map(scipy.sparse.load_npz, files.adjacency_filenames(NUM_PARTITIONS))
        ])
        next_rank += edge_follow_prob * spread_probs[:, 0]  # make column vector 1-D
        # update destinations from sink nodes
        next_rank[:] += edge_follow_prob * current_rank[out_degree == 0].sum() / N
        # copy `next_rank` values into `current_rank``
        current_rank[:] = next_rank
        # --
        # compute perplexity and progress
        prev_perplexity = current_perplexity
        current_perplexity = perplexity(current_rank)
        next_iter_start = time.time()
        print("{:<3d}    {:<15.6f}   {:.3f}".format(current_iter,
                                                    current_perplexity - prev_perplexity,
                                                    next_iter_start - iter_start))
        iter_start = next_iter_start
    df = pl.DataFrame({
        "title": id_map.keys(), "value": next_rank, "in_deg": in_degree, "out_deg": out_degree,
    })
    return df

In [31]:
# Run until perplexity changes by less than 1
PR = personalized_page_rank(log_out_degree)

Itr# | ΔPerplexity     | Seconds
1      -6013796.145013   6.664
2      145977.970099     4.352
3      -38511.688381     3.560
4      -2270.893838      3.486
5      -8357.308689      3.434
6      -2717.844135      3.383
7      -3166.973678      3.376
8      -1640.290433      3.345
9      -1428.332263      3.334
10     -892.535524       3.370
11     -699.347582       3.386
12     -469.568557       3.320
13     -357.639085       3.311
14     -246.562236       3.303
15     -186.548257       3.304
16     -130.940713       3.338
17     -98.650553        3.296
18     -70.132001        3.306
19     -52.927872        3.314
20     -37.852054        3.279
21     -28.705336        3.289
22     -20.618140        3.291
23     -15.707255        3.288
24     -11.320650        3.305
25     -8.670001         3.291
26     -6.260557         3.283
27     -4.822257         3.265
28     -3.486686         3.308
29     -2.700787         3.254
30     -1.954268         3.287
31     -1.522622         3.266
32    

In [32]:
PR_sorted = PR.sort('value', descending=True)

In [33]:
pager(PR_sorted.slice(0, 2000), 20)

interactive(children=(Dropdown(description='page', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …

<function nbhelpers.polars.pager.<locals>.<lambda>(page)>

In [34]:
searcher(
    PR_sorted.slice(0, 200000).with_columns(pl.Series("rank", range(200000))).select(["rank", *PR_sorted.columns]),
    ['title'],
    20
)

interactive(children=(Text(value='', description='q'), Output()), _dom_classes=('widget-interact',))

<function nbhelpers.polars.searcher.<locals>.searcher_run(q)>

In [35]:
PR.write_parquet(files.pagerank_parquet_filename)